In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import nltk 
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

data = gutenberg.raw('shakespeare-hamlet.txt')
emma_text = gutenberg.raw('austen-emma.txt')
moby_dick = gutenberg.raw('melville-moby_dick.txt')
##save to file

with open('hamlet.txt', 'w', encoding='utf-8') as f:
    f.write(data)

with open('emma.txt', 'w', encoding='utf-8') as f:
    f.write(emma_text)


[nltk_data] Downloading package gutenberg to /usr/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [10]:
with open('combined_dataset.txt', 'w', encoding='utf-8') as f:
    f.write(data + '\n' + emma_text + '\n' + moby_dick)

print("✅ All texts saved! You can now train on 'combined_dataset.txt'")


✅ All texts saved! You can now train on 'combined_dataset.txt'


In [12]:
## Data Preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

##load The Dataset
with open('/kaggle/working/combined_dataset.txt','r') as file:
    text = file.read().lower()

words = text.split()
limited_text = ' '.join(words[:100000])  # first 100,000 words only
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

22711

In [13]:
input_sequences = []
max_seq_len = 20
max_total_sequences = 50000  # Optional: limit to prevent RAM crash

for line in limited_text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Skip very short lines
    if len(token_list) < 2:
        continue

    # Slide through the token list with a step of 1 or 20 (you can adjust this)
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]

        # Break long sequences into chunks of 20
        if len(n_gram_seq) > max_seq_len:
            for j in range(max_seq_len, len(n_gram_seq) + 1, max_seq_len):
                chunk = n_gram_seq[j - max_seq_len:j]
                input_sequences.append(chunk)
        else:
            input_sequences.append(n_gram_seq)

        if len(input_sequences) >= max_total_sequences:
            break

    if len(input_sequences) >= max_total_sequences:
        break


In [14]:
input_sequences[0:20]

[[1, 6133],
 [1, 6133, 3],
 [1, 6133, 3, 398],
 [1, 6133, 3, 398, 30],
 [1, 6133, 3, 398, 30, 1407],
 [1, 6133, 3, 398, 30, 1407, 9334],
 [1, 6133, 3, 398, 30, 1407, 9334, 12829],
 [1, 6133, 3, 398, 30, 1407, 9334, 12829, 9335],
 [1, 6133, 3, 398, 30, 1407, 9334, 12829, 9335, 12830],
 [1, 6133, 3, 398, 30, 1407, 9334, 12829, 9335, 12830, 12831],
 [1, 6133, 3, 398, 30, 1407, 9334, 12829, 9335, 12830, 12831, 12832],
 [1, 6133, 3, 398, 30, 1407, 9334, 12829, 9335, 12830, 12831, 12832, 386],
 [1,
  6133,
  3,
  398,
  30,
  1407,
  9334,
  12829,
  9335,
  12830,
  12831,
  12832,
  386,
  3760],
 [1,
  6133,
  3,
  398,
  30,
  1407,
  9334,
  12829,
  9335,
  12830,
  12831,
  12832,
  386,
  3760,
  2],
 [1,
  6133,
  3,
  398,
  30,
  1407,
  9334,
  12829,
  9335,
  12830,
  12831,
  12832,
  386,
  3760,
  2,
  9336],
 [1,
  6133,
  3,
  398,
  30,
  1407,
  9334,
  12829,
  9335,
  12830,
  12831,
  12832,
  386,
  3760,
  2,
  9336,
  110],
 [1,
  6133,
  3,
  398,
  30,
  1407,
  

In [15]:
## padding the sequences
max_sequence_length = max([len(x) for x in input_sequences])
max_sequence_length

20

In [16]:
input_sequences = np.array(pad_sequences(input_sequences,maxlen = max_sequence_length,padding = 'pre'))
input_sequences

array([[    0,     0,     0, ...,     0,     1,  6133],
       [    0,     0,     0, ...,     1,  6133,     3],
       [    0,     0,     0, ...,  6133,     3,   398],
       ...,
       [  127,   984,  1135, ...,    20,    38, 12930],
       [    2,    38,  1877, ...,   920,     2, 12932],
       [  369,     4,   409, ...,  7365,   353,    19]], dtype=int32)

In [17]:
##creating input and label

import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]

y

array([ 6133,     3,   398, ..., 12930, 12932,    19], dtype=int32)

In [18]:
y = tf.keras.utils.to_categorical(y,num_classes = total_words)

In [19]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
#Define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length = max_sequence_length))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(total_words,activation="softmax"))

##compiling
model.build(input_shape=(None, max_sequence_length))
model.compile(loss = "categorical_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [24]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 100)        │     2,271,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 20, 128)        │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 22711)          │     2,929,719 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,449,651 (20.79 MB)

 Trainable params: 5,449,651 (20.79 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
##training
history = model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test),verbose = 1)

Epoch 1/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 25s 17ms/step - accuracy: 0.0746 - loss: 4.4032 - val_accuracy: 0.0770 - val_loss: 3.8128
Epoch 2/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.0953 - loss: 3.7013 - val_accuracy: 0.7970 - val_loss: 1.0833
Epoch 3/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.8846 - loss: 0.6756 - val_accuracy: 0.9797 - val_loss: 0.1483
Epoch 4/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.9839 - loss: 0.1184 - val_accuracy: 0.9997 - val_loss: 0.0423
Epoch 5/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.9994 - loss: 0.0341 - val_accuracy: 0.9997 - val_loss: 0.0146
Epoch 6/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.9996 - loss: 0.0123 - val_accuracy: 0.9997 - val_loss: 0.0076
Epoch 7/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.9996 - loss: 0.0066 - val_accuracy: 0.9997 - val_loss: 0.0064
Epoch 8/10
1251/1251 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.9996 -

In [26]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_next_word(model, tokenizer, text, max_sequence_length):
    # Convert input text to token list
    token_list = tokenizer.texts_to_sequences([text])[0]

    # Ensure the token list is of the correct length
    if len(token_list) >= max_sequence_length:
        token_list = token_list[-(max_sequence_length - 1):]  # leave space for next word

    # Pad the sequence
    token_list = pad_sequences([token_list], maxlen=max_sequence_length, padding='pre')

    # Predict the next word
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted_probs, axis=1)[0]  # Get scalar from array

    # Map index back to word
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word

    return None  # In case the word wasn't found


In [27]:
input_text = "To be or not to be"
print(f"Input_text :{input_text}")

Input_text :To be or not to be


In [28]:
max_sequence_len = model.input_shape[1]
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"next word prediction :{next_word}")

next word prediction :clad


In [29]:
model.save("next_word_lstm.h5")

In [30]:
##save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
 pickle.dump(tokenizer,handle,protocol = pickle.HIGHEST_PROTOCOL)